# Face Detection and Recognition

Create a script in OpenCV-Python that recognizes from a video stream (either webcam or saved video file) and frames your own face in real time! So-called "false positives" should be avoided, ie if your own face is missing from the video stream, the program should not point out anyone else's face.

* First detect all faces in each frame, including by Haar-Cascading (Viola-Jones algorithm) 
* Apply LBPH (Low Binary Pattern Histograms) to identify your face

In [1]:
# Import libraries
import os
import numpy as np
import cv2 as cv
from PIL import Image 

### Test for video feed that it works

cap = cv.VideoCapture(0) can be changed to cap = cv2.VideoCapture("<Video_Name_Here.mp4>") to use a video output


In [4]:
# Test Camera video feed
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')

cap = cv.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Draw frame around detected faces
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.5, 5)
    for (x,y,w,h) in faces:
        cv.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)

    # Display the resulting frame
    cv.imshow('Frame',frame)
    
    # Quit when ESC-key is pressed
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

# When everything done, release the capture
cap.release()
cv.destroyAllWindows()

## Script for taking test images of your face for the dataset

Uses the video feed to save images off your face. Saves them to folder dataset

In [20]:
# Save images of face for training
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')

cap = cv.VideoCapture(0)

faceNumber = 0 # Incremented number for images

while(True):

    ret, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.5, 5)

    #Save faces that are in frame
    for (x,y,w,h) in faces:
        faceNumber += 1
        cv.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2) # Show user when images are taken
        cv.imwrite("./dataSet/1."+str(faceNumber)+ ".jpg", gray[y:y+h, x:x+w])
    
    cv.imshow('img',frame)
    
    # Quit when ESC-key is pressed
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
        
# When everything done, release the capture        
cap.release()
cv.destroyAllWindows()

## Script for training face recognition with the dataset 

In [21]:
recognizer = cv.face.LBPHFaceRecognizer_create()
detector= cv.CascadeClassifier("haarcascade_frontalface_default.xml");

def getImagesAndLabels(path):
    #get the path of all the files in the folder
    imagePaths=[os.path.join(path,f) for f in os.listdir(path)] 
    #create empth face list
    faceSamples=[]
    #create empty ID list
    Ids=[]
    
    #now looping through all the image paths and loading the Ids and the images
    for imagePath in imagePaths:
        #loading the image and converting it to gray scale
        pilImage=Image.open(imagePath).convert('L')
        #Now we are converting the PIL image into numpy array
        imageNp=np.array(pilImage,'uint8')
        #getting the Id from the image
        Id=int(os.path.split(imagePath)[1].split(".")[0])
        # extract the face from the training image sample
        faces=detector.detectMultiScale(imageNp)
        #If a face is there then append that in the list as well as Id of it
        for (x,y,w,h) in faces:
            faceSamples.append(imageNp[y:y+h,x:x+w])
            Ids.append(Id)
    return faceSamples,Ids


faces,Ids = getImagesAndLabels('./dataSet')
recognizer.train(faces, np.array(Ids))
recognizer.save('trainner.yml')


## Script for my facial detection

The conf variable should be changed depending on the trained dataset. In my case the confidence is around 50-65.

In [27]:
# Face detection

face_cascade = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')

# Video feed
cap = cv.VideoCapture(0)
#cap = cv.VideoCapture("head-pose-face-detection-male.mp4")

# Recognizer
recognizer = cv.face.LBPHFaceRecognizer_create()
recognizer.read("trainner.yml")
id=0

# Font variables
font =  cv.FONT_HERSHEY_PLAIN
fontSize = 1.0
fontColor = (0, 0, 0) 
fontThickness = 1

while 1:

    ret, img = cap.read()
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.5, 5)

    for (x,y,w,h) in faces:
        #cv.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        id,conf=recognizer.predict(gray[y:y+h,x:x+w])
        #print(100*(1-(conf)/300))
        print(conf)
        
        # If confidence is under 65 the face should be a great match with Mine
        if( id == 1 and conf<65):
            cv.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            cv.putText(img,"Niklas",(int(x+(w/3)) ,y+h),font, fontSize, fontColor, fontThickness)

    
    cv.imshow('img',img)

    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv.destroyAllWindows()

78.73082781343058
75.9786332293736
78.20384397228395
73.32064455784209
70.6768040284789
76.32857425123736
58.57258358969239
72.28679281614338
60.03427442453976
64.05382775365092
78.63257822470277
64.03265097429184
60.94108502456873
63.18991774270694
69.91379677527358
71.23086575113805
74.62851169667077
76.33078638655485
77.61512634529139
77.17563095492882
78.73650760673775
77.24545594357974
78.95213429794046
81.62657585042933
79.93347002328798
82.84747091977871
85.95488846811381
79.53431015999561
84.30531908342044
81.7334031080469
80.04065510425103
79.24804233299209
79.44448251356195
80.47410325706885
79.55008253052304
78.40545434126216
78.0184568642798
81.36436325831441
81.59586374757936
80.18577068207883
80.59249750854126
79.65072435345327
76.44342124796083
75.43768712764064
75.50944855236546
68.90941192778665
70.09069814490424
68.02912947610926
66.94187267462009
67.95195987480695
68.99529605509592
70.10453072597043
67.7291454987466
67.28544614407866
68.84566547297575
64.278644567385

## Script for saving the demo video

In [24]:
# Face detection

face_cascade = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')

# Video feed
cap = cv.VideoCapture(0)
#cap = cv.VideoCapture("head-pose-face-detection-male.mp4")

# Define the codec and create VideoWriter object
fourcc = cv.VideoWriter_fourcc(*'XVID')
(grabbed, frame) = cap.read() # Get camera width and height
fshape = frame.shape
out = cv.VideoWriter('output.mp4',fourcc, 12.0, (fshape[1],fshape[0])) # remeber to keep resolution in input and output same

# Recognizer
recognizer = cv.face.LBPHFaceRecognizer_create()
recognizer.read("trainner.yml")
id=0

# Font variables
font =  cv.FONT_HERSHEY_PLAIN
fontSize = 1.0
fontColor = (0, 0, 0) 
fontThickness = 1

while 1:

    ret, img = cap.read()
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.5, 5)

    for (x,y,w,h) in faces:
        #cv.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        id,conf=recognizer.predict(gray[y:y+h,x:x+w])
        #print(100*(1-(conf)/300))
        #print(conf)
        
        # If confidence is under 65 the face should be a great match with dataset
        if( id == 1 and conf<65):
            cv.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            cv.putText(img,"Niklas",(int(x+(w/3)) ,y+h),font, fontSize, fontColor, fontThickness)

    
    cv.imshow('img',img)
    out.write(img)

    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

out.release()        
cap.release()
cv.destroyAllWindows()